#Data Extraction
### Here we have "dialects_database.db"

In [20]:
import pandas as pd
import sqlite3

In [52]:
db = '/content/drive/MyDrive/dialects_database.db'
csv_out_path = '/content/drive/MyDrive/Data.csv'

In [31]:
def connect(db, is_cursure_needed=False):
  # connect to the database using "sqlit3"
  conn = sqlite3.connect(db)
  if is_cursure_needed:
      # create a cursor object to execute queries
      cursor = conn.cursor()
      return conn, cursor
  else:
      return conn

##let's browse the data and see what it contains

In [32]:
conn, cursor = connect(db, True)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables_names = cursor.fetchall()
for table_name in tables_names:
    cursor.execute(f"SELECT name FROM PRAGMA_TABLE_INFO('{table_name[0]}')")
    print(f'Table: {table_name[0]}\n Has columns: {cursor.fetchall()}')
cursor.close()
conn.close()

Table: id_text
 Has columns: [('id',), ('text',)]
Table: id_dialect
 Has columns: [('id',), ('dialect',)]


the output told use that there is two tables one contains the text and the other table contains the label so we need to merge these to tables on ID

## Extraction to "DataFrame"

From the above inruation i will use pandas to read data from the data base according to certain query then save the data using bandas

In [35]:
conn = connect(db)

query = '''
SELECT T2.text, T1.dialect
FROM id_dialect AS T1
JOIN id_text AS T2 ON T1.ID = T2.ID
'''

text_dialect_DF = pd.read_sql_query(query, conn)

We need to ensure that we retrieved the whole data and not miss anything

In [50]:
query = "SELECT COUNT(text) AS c FROM id_text"

if pd.read_sql_query(query, conn)['c'][0] == len(text_dialect_DF):
    print("Congratulations all the data were retrieved:)")
else:
    print("Sadly there is a missing data :(")

Congratulations all the data were retrieved:)


In [51]:
conn.close()

##As all the data retrieved let's save it to a csv file

In [53]:
text_dialect_DF.to_csv(csv_out_path)

In [55]:
df = pd.read_csv(csv_out_path)
df.read_csv

ParserError: ignored